In [1]:
import pandas as pd
import ast
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import classification_report

# Predict on the original splits

In [2]:
df = pd.read_csv("data/preprocessed_data.csv")
df["joined_tokens"] = df["tokens"].apply(ast.literal_eval).apply(lambda toks: " ".join(toks))

df.head()

,sentence,topic,sentiment,dataset,tokens,joined_tokens
0,slide giáo trình đầy đủ .,1,2,train,"['slide', 'giáo_trình', 'đầy_đủ']",slide giáo_trình đầy_đủ
1,"nhiệt tình giảng dạy , gần gũi với sinh viên .",0,2,train,"['nhiệt_tình', 'giảng_dạy', 'gần_gũi', 'sinh_v...",nhiệt_tình giảng_dạy gần_gũi sinh_viên
2,đi học đầy đủ full điểm chuyên cần .,1,0,train,"['đi', 'học', 'đầy_đủ', 'full', 'chuyên_cần']",đi học đầy_đủ full chuyên_cần
3,chưa áp dụng công nghệ thông tin và các thiết ...,0,0,train,"['áp_dụng', 'công_nghệ', 'thông_tin', 'thiết_b...",áp_dụng công_nghệ thông_tin thiết_bị giảng_dạy
4,"thầy giảng bài hay , có nhiều bài tập ví dụ ng...",0,2,train,"['thầy', 'giảng', 'bài_tập', 'ví_dụ', 'lớp']",thầy giảng bài_tập ví_dụ lớp


In [3]:
def get_data_for_task(df, task):
    df_train = df[df["dataset"] == "train"]
    df_val   = df[df["dataset"] == "val"]
    df_test  = df[df["dataset"] == "test"]

    X_train = df_train["joined_tokens"].tolist()
    y_train = df_train[task].tolist()

    X_val = df_val["joined_tokens"].tolist()
    y_val = df_val[task].tolist()

    X_test = df_test["joined_tokens"].tolist()
    y_test = df_test[task].tolist()

    return X_train, y_train, X_val, y_val, X_test, y_test

In [4]:
def run_model(name, model, vectorizer, X_train, y_train, X_test, y_test):
    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec  = vectorizer.transform(X_test)

    model.fit(X_train_vec, y_train)
    y_pred = model.predict(X_test_vec)

    print(f"{name}")
    print(classification_report(y_test, y_pred, digits=4))
    print("-" * 60)

In [5]:
for task in ["topic", "sentiment"]:
    print(f"\n====== TASK: {task.upper()} ======\n")
    X_train, y_train, X_val, y_val, X_test, y_test = get_data_for_task(df, task)

    run_model(
        name="MultinomialNB + CountVectorizer",
        model=MultinomialNB(),
        vectorizer=CountVectorizer(),
        X_train=X_train,
        y_train=y_train,
        X_test=X_test,
        y_test=y_test
    )

    run_model(
        name="SVC (linear) + TfidfVectorizer",
        model=SVC(kernel="linear"),
        vectorizer=TfidfVectorizer(),
        X_train=X_train,
        y_train=y_train,
        X_test=X_test,
        y_test=y_test
    )


====== TASK: TOPIC ======

MultinomialNB + CountVectorizer
              precision    recall  f1-score   support

           0     0.8590    0.9393    0.8974      2290
           1     0.6660    0.5927    0.6272       572
           2     0.9122    0.9310    0.9215       145
           3     0.6000    0.0189    0.0366       159

    accuracy                         0.8301      3166
   macro avg     0.7593    0.6205    0.6207      3166
weighted avg     0.8136    0.8301    0.8064      3166

------------------------------------------------------------
SVC (linear) + TfidfVectorizer
              precision    recall  f1-score   support

           0     0.8818    0.9288    0.9047      2290
           1     0.6591    0.6626    0.6609       572
           2     0.8904    0.8966    0.8935       145
           3     0.6364    0.1321    0.2188       159

    accuracy                         0.8392      3166
   macro avg     0.7669    0.6550    0.6694      3166
weighted avg     0.8297    0.8392

# Predict on the resplitted dataset

In [6]:
df = pd.read_csv("data/resplitted_data.csv")
df["joined_tokens"] = df["tokens"].apply(ast.literal_eval).apply(lambda toks: " ".join(toks))

df.head()

,Unnamed: 0,sentence,topic,sentiment,tokens,dataset,joined_tokens
0,0,"dạy dễ hiểu , thân thiện .",0,2,"['dạy', 'thân_thiện']",train,dạy thân_thiện
1,1,"giáo viên thực hành cần năng động hơn , thầy c...",0,0,"['giáo_viên', 'thực_hành', 'năng_động', 'thầy'...",train,giáo_viên thực_hành năng_động thầy vẻ hơi thụ_...
2,2,giảng viên cần doubledot cho bài tập thêm về n...,0,0,"['giảng_viên', 'doubledot', 'bài_tập', 'ngữ_ph...",train,giảng_viên doubledot bài_tập ngữ_pháp
3,3,thầy dạy tốt và rất nhiệt tình .,0,2,"['thầy', 'dạy', 'nhiệt_tình']",train,thầy dạy nhiệt_tình
4,4,em rất hài lòng về phương pháp dạy này .,0,2,"['hài_lòng', 'phương_pháp', 'dạy']",train,hài_lòng phương_pháp dạy


In [7]:
for task in ["topic", "sentiment"]:
    print(f"\n====== TASK: {task.upper()} ======\n")
    X_train, y_train, X_val, y_val, X_test, y_test = get_data_for_task(df, task)

    run_model(
        name="MultinomialNB + CountVectorizer",
        model=MultinomialNB(),
        vectorizer=CountVectorizer(),
        X_train=X_train,
        y_train=y_train,
        X_test=X_test,
        y_test=y_test
    )

    run_model(
        name="SVC (linear) + TfidfVectorizer",
        model=SVC(kernel="linear"),
        vectorizer=TfidfVectorizer(),
        X_train=X_train,
        y_train=y_train,
        X_test=X_test,
        y_test=y_test
    )


====== TASK: TOPIC ======

MultinomialNB + CountVectorizer
              precision    recall  f1-score   support

           0     0.8582    0.9436    0.8988      2321
           1     0.6875    0.5970    0.6391       608
           2     0.8811    0.8811    0.8811       143
           3     0.4167    0.0307    0.0571       163

    accuracy                         0.8297      3235
   macro avg     0.7109    0.6131    0.6190      3235
weighted avg     0.8048    0.8297    0.8068      3235

------------------------------------------------------------
SVC (linear) + TfidfVectorizer
              precision    recall  f1-score   support

           0     0.8735    0.9431    0.9070      2321
           1     0.7205    0.6530    0.6851       608
           2     0.8493    0.8671    0.8581       143
           3     0.7500    0.1472    0.2462       163

    accuracy                         0.8451      3235
   macro avg     0.7983    0.6526    0.6741      3235
weighted avg     0.8375    0.8451